In [1]:
import numpy as np
import random
import os
from scipy.misc import imsave, imread
from os import listdir
from os.path import isfile, join

%matplotlib inline

In [2]:
import sys
import tarfile
from PIL import Image 
from scipy import ndimage
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [13]:
from sklearn.model_selection import train_test_split
from math import ceil

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D,\
    Dropout, Flatten, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

import matplotlib.pyplot as plt

Using Theano backend.


In [4]:
import shutil

In [3]:
def to_rgb2(im):
    w, h = im.shape
    ret = np.empty((w, h, 3), dtype=np.uint8)
    ret[:, :, :] = im[:, :, np.newaxis]
    return ret

In [9]:
# MOVE ALL FILES TO ONE DIRECTORY

rootdir = '.train_resized'
moveto = './X_train'
i = 0
for subdir, dirs, files in os.walk(rootdir):
    if(subdir  == './train_resized'):
        continue
        
    for i in files:
        shutil.move(subdir + '/' + i,moveto)

In [5]:
def check_class(l):
    l = l[:3]
    if l[0] == '0':
        if l[1]=='0':
            l=l[2:]
        else:
            l= l[1:]
    return l
d = check_class('009_345.jpg')
print d

9


In [21]:
from_train = './X_train'

files = [f for f in listdir(from_train) if isfile(join(from_train, f))]

l = []
for j in range(1, 257):
    l.append(j)

for i in l:
    check_10 = 0
    for image in files:
        if check_10 < 10 and int(check_class(image)) == i:
            fromdest = "./X_train/"+image
            todest = "./X_val/"+image
            shutil.move(fromdest, todest)
            check_10 += 1


In [9]:
path_val = './X_val'
files_val = [f for f in listdir(path_val) if isfile(join(path_val, f))]

y_file = open("y_val_64.txt", 'w')
for im in files_val:
    class_ =check_class(im) 
    y_file.write(im+ ' '+ class_+'\n')
y_file.close()

In [6]:
mypath = './X_train'
files_tr = [f for f in listdir(mypath) if isfile(join(mypath, f))]

y_file = open("y_train_64.txt", 'w')
for im in files_tr:
    class_ =check_class(im) 
    y_file.write(im+ ' '+ class_+'\n')
y_file.close()

In [8]:
def load_images(subset):
    path_to_images = subset
    labels_file = './train_labels.txt'
    image_labels = {}
    
    with open(labels_file, 'r') as f:
        dict_labels = dict([line.strip().split() for line in f.readlines()])
    files = os.listdir(path_to_images)

    # Avoid hidden files
    files = filter(lambda files: not files.startswith('.'), files)

    images = np.zeros((len(files), 64, 64,3), dtype=np.uint8)
    labels = np.zeros(len(files), dtype=np.uint8)
    
    q = 0
    all_files = []
    for fid, file in enumerate(files):
        if fid % 1000 == 0:
            print fid
        image = imread(path_to_images + '/' + file)
        all_files.append(file)
        if image.shape == (64, 64):
            image = to_rgb2(image)
            
        images[fid] = image
        labels[fid] = int(dict_labels[file])
    print "Done!"
    return images,labels, files, all_files


#print "loading train images..."
X_train, y_train, train_files, list_images = load_images('./train_resized')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
Done!


In [9]:
X_train.shape

(19540, 64, 64, 3)

In [10]:
y_train.shape

(19540,)

In [11]:
def load_images_test(subset):
    path_to_images = subset
    files = os.listdir(path_to_images)
    files = filter(lambda files: not files.startswith('.'), files)

    images = np.zeros((len(files), 64, 64,3), dtype=np.uint8)
    all_files = []
    for fid, file in enumerate(files):
        if fid % 1000 == 0:
            print fid
        image = imread(path_to_images + '/' + file)
        all_files.append(file)
        if image.shape == (64, 64):
            image = to_rgb2(image)
            
        images[fid] = image
    print "Done!"
    return images, files, all_files

X_test, test_files, list_test = load_images_test('./test_resized')

0
1000
2000
3000
4000
5000
6000
7000
Done!


In [12]:
def load_images_validation(subset):
    path_to_images = subset
    labels_file = '....path to labels.../validation_labels.txt'
    image_labels = {}
    
    with open(labels_file, 'r') as f:
        dict_labels = dict([line.strip().split() for line in f.readlines()])
    files = os.listdir(path_to_images)
    
    # Avoid hidden files
    files = filter(lambda files: not files.startswith('.'), files)
    
    images = np.zeros((len(files), 64, 64,3), dtype=np.uint8)
    labels = np.zeros(len(files), dtype=np.uint8)
    
    q = 0
    all_files = []
    for fid, file in enumerate(files):
        if fid % 1000 == 0:
            print fid
        image = imread(path_to_images + '/' + file)
        all_files.append(file)
        if image.shape == (64, 64):
            image = to_rgb2(image)
            
        images[fid] = image
        labels[fid] = int(dict_labels[file])
    print "Done!"
    return images,labels, all_files


#print "loading train images..."
X_val, y_val, all_files = load_images_validation('./validation_resized')

0
1000
2000
Done!


## Classifier

### Validation split

In [81]:
X_val = X_train[:4000]
X_tr = X_train[4001:]
y_val = y_train[:4000]
y_tr = y_train[4001:]

In [82]:
X_tr.shape

(18896, 64, 64, 3)

In [83]:
# ------------- PREPROCESSING --------------------- 

X_tr = X_tr.astype('float32')
X_test = X_test.astype('float32')
X_tr /= 255.0
X_test /= 255.0
X_test -= X_tr.mean(0)
X_test /= X_tr.std(0)

X_tr -= X_tr.mean(0)
X_tr /= X_tr.std(0)
X_val = X_val.astype('float32')
X_val /=255.0
X_val -= X_tr.mean(0)
X_val /= X_tr.std(0)

In [14]:
input_shape = (64, 64, 3) 
num_classes = 256

### Simple CNN

In [15]:
model = Sequential([
    Conv2D(16, (5, 5), strides=(2, 2), padding='same', kernel_initializer='he_uniform', use_bias=False, 
        input_shape=input_shape), 
    BatchNormalization(scale=False),
    Activation('relu'),
    
    Conv2D(32, (3, 3), padding='same', kernel_initializer='he_uniform', use_bias=False),   
    BatchNormalization(scale=False),
    Activation('relu'),
    
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),  
    Dropout(0.2),

    Conv2D(32, (1, 1), padding='same', kernel_initializer='he_uniform', use_bias=False), 
    BatchNormalization(scale=False),
    Activation('relu'),
    
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'), 
    Dropout(0.2),
    
    Flatten(), 
    Dense(512, kernel_initializer='he_uniform'), 
    Activation('relu'), 
    Dropout(0.2),
    Dense(num_classes, kernel_initializer='he_uniform'), 
    Activation('softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer=keras.optimizers.Adam(lr=0.001),
    metrics=['accuracy']
)

model.count_params()

1187488

In [17]:
batch_size = 128
y_train = keras.utils.to_categorical(y_train, num_classes)

In [18]:
y_val = keras.utils.to_categorical(y_val, num_classes)

In [20]:
model.fit(X_train, y_train, batch_size=128, epochs=20, validation_data=(X_val, y_val))

Train on 19540 samples, validate on 2560 samples
Epoch 1/20
19540/19540 [==============================] - 140s - loss: 5.1094 - acc: 0.0994 - val_loss: 5.4113 - val_acc: 0.0277

In [40]:
predicted_labels = model.predict_classes(X_test)

7680/7680 [==============================] - 16s     

In [74]:
for i in range(len(predicted_labels)):
    if predicted_labels[i] == 20:
        print list_test[i]

28673.jpg
4357.jpg
7286.jpg
12160.jpg
15925.jpg
7134.jpg
28903.jpg
10059.jpg
23056.jpg
7275.jpg
14924.jpg
2773.jpg
17829.jpg
32487.jpg
